<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

batch_size = 128
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


def remove_numbers(x,y,list_of_numbers_to_keep):
    y_df = pd.DataFrame(y,columns=['Label'])
    y_df = y_df[y_df['Label'].apply(str).isin(list_of_numbers_to_keep)]
    x = x[y_df.index,:,:,:]
    y = y_df.to_numpy()
    return x,y.flatten()

num_to_keep = ['0','1']
num_classes = len(num_to_keep)
x_train,y_train = remove_numbers(x_train,y_train,num_to_keep)
x_test,y_test = remove_numbers(x_test,y_test,num_to_keep)

# # convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
    

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
def baseline_model(name,num_classes=2):
    model = Sequential(name=name)
    model.add(Conv2D(2, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(7, 7)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

baseline_model('test').summary()

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=(28,28,1)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

In [ ]:
import random
rand_numbers = random.sample(range(0, 1000), 100)
rand_numbers
x_test2 = x_test[rand_numbers,:,:,:]
y_test2 = y_test[rand_numbers,:]

In [80]:
from utils import *

In [ ]:
train_model_iteratively(baseline_model,x_train,y_train,x_test2,y_test2,'./data/tmp_cnn')

In [ ]:
yDF = pd.DataFrame(y_test2,columns=['Actual Label 0','Actual Label 1'])
yDF['point']=range(100)
y_pred = pd.read_csv('./data/tmp_cnn/predictions.csv')
y_pred.rename({y_pred.columns[0]:'point'},axis=1,inplace=True)
y_pred2 = pd.merge(yDF,y_pred,left_on='point',right_on='point')
y_pred2.to_csv('./data/tmp_cnn/predictions2.csv')

In [73]:
model_weights=get_model_weights('./data/tmp_cnn')

model.1-1-0.44.hdf5
model.1-3-0.41.hdf5
model.0-3-2.10.hdf5
model.0-1-7.67.hdf5


In [78]:
model_weights[list(model_weights.keys())[0]]['conv_kernel_0'].flatten().shape

(18,)

In [89]:
model_weights[list(model_weights.keys())[0]]['conv_kernel_0'].shape

(3, 3, 1, 2)

In [71]:
def get_kernel_names(f):
    datalist = []
    def returnname(name):
        if 'kernel' in name and name not in datalist and 'model_weights' in name:
            return name
        else:
            return None
    looper = 1
    while looper == 1:
        name = f.visit(returnname)
        if name == None:
            looper = 0
            continue
        datalist.append(name)
    return datalist


In [72]:
get_kernel_names(f)

['model_weights/conv2d_25/conv2d_25/kernel:0',
 'model_weights/dense_25/dense_25/kernel:0']

In [88]:
convert_weight_dict_to_dataframe(model_weights)

,model_id,epoch,val_loss,conv_kernel_0_0,conv_kernel_0_1,conv_kernel_0_2,conv_kernel_0_3,conv_kernel_0_4,conv_kernel_0_5,conv_kernel_0_6,...,dense_kernel_0_26,dense_kernel_0_27,dense_kernel_0_28,dense_kernel_0_29,dense_kernel_0_30,dense_kernel_0_31,dense_kernel_0_32,dense_kernel_0_33,dense_kernel_0_34,dense_kernel_0_35
model.1-1-0.44.hdf5,model.1,1,0.44,-0.036989,-0.139469,0.252992,-0.518346,0.196239,0.470156,0.056776,...,0.275780,-0.454845,-0.001757,-0.323857,-0.320206,0.364231,-0.240478,-0.215816,0.647348,-0.259712
model.1-3-0.41.hdf5,model.1,3,0.41,-0.094851,-0.145225,0.202130,-0.545969,0.150403,0.431502,-0.001046,...,0.271950,-0.451015,-0.042575,-0.283039,-0.290661,0.334686,-0.238388,-0.217907,0.691822,-0.304186
model.0-3-2.10.hdf5,model.0,3,2.10,0.030580,0.333061,0.194057,-0.065567,-0.467988,-0.388728,0.278404,...,0.715139,-0.385276,0.434537,0.030749,0.315552,0.290995,0.054417,-0.053401,0.129896,0.292715
model.0-1-7.67.hdf5,model.0,1,7.67,0.080272,0.411834,0.235749,-0.021754,-0.448223,-0.357529,0.328421,...,0.695996,-0.366132,0.455155,0.010131,0.238270,0.368277,0.055963,-0.054947,0.089073,0.333538
